In [15]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
np.random.seed(521)
tf.random.set_seed(1314)
pi = tf.constant(np.pi, dtype=tf.float64)
class HeatPINN(keras.Sequential):
    def __init__(self, Layers, name=None):
        super(HeatPINN, self).__init__(name=name)
        self.add(keras.Input(shape=(Layers[0],), dtype=tf.float64))
        for i in range(1, len(Layers)-1):
            self.add(keras.layers.Dense(Layers[i], dtype=tf.float64, activation='tanh'))
        self.add(keras.layers.Dense(Layers[-1], dtype=tf.float64, name="outputs"))
    
    @tf.function
    def loss_U(self, X_u_train, u_train):
        u = self(X_u_train)
        loss_u = tf.reduce_mean( tf.square(u_train - u))
        return loss_u
    
    @tf.function
    def loss_PDE(self, X_f_train,X_b_train):
        x = X_f_train[:, 0:1]
        y = X_f_train[:, 1:2]
        with tf.GradientTape(persistent=True) as tape:
            tape.watch([x, y])
            X = tf.concat([x, y], axis=1)
            u = self(X)
            u_x = tape.gradient(u, x)
            u_y = tape.gradient(u, y)
            u_xx = tape.gradient(u_x, x)
            u_yy = tape.gradient(u_y, y)
        del tape
        u_i=self(X_b_train)
        loss_f1= u_xx + u_yy +2*pi**2*tf.sin(pi*x)*tf.sin(pi*y)
        loss_f1 = tf.reduce_mean(tf.square(loss_f1))
        loss_f2 =tf.reduce_mean(tf.square(u_i))
        loss_f = loss_f1 + loss_f2
        return loss_f
    
    @tf.function
    def train_step(self, X_u_train, u_train, X_i_train):
        with tf.GradientTape() as tape:
            loss = self.loss_U(X_u_train, u_train) + self.loss_PDE(X_u_train,X_i_train)
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return loss
    
    def train(self, X_u_train, u_train, X_i_train, epochs=200):
        
        for epoch in tf.range(1, epochs+1):
            loss = self.train_step(X_u_train, u_train, X_i_train)
            if epoch % 50 == 0:
                tf.print("Training loss (every 50 epochs) at epoch", epoch, ":", loss)
                loss_per50.append(loss)
# 定义迪利克雷边界条件热传导方程的参数和边界条件函数
u_exact = lambda x, y: np.sin(np.pi * x) * np.sin(np.pi * y)  # 精确解

# 生成训练数据
loss_per50=[]
n_u = 5000 # 内部节点数量
n_i = 5000  # 边界节点数量
X_u_train = np.random.uniform(low=0, high=1, size=(n_u, 2))
u_train = u_exact(X_u_train[:, 0:1], X_u_train[:, 1:2])
X_i_train_1 = np.random.uniform(low=0, high=1, size=(1250, 2))
X_i_train_1[:, 0] = 1
X_i_train_2 = np.random.uniform(low=0, high=1, size=(1250, 2))
X_i_train_2[:, 0] = 0
X_i_train_3 = np.random.uniform(low=0, high=1, size=(1250, 2))
X_i_train_3[:, 1] = 1
X_i_train_4 = np.random.uniform(low=0, high=1, size=(1250, 2))
X_i_train_4[:, 1] = 0
X_i_train = np.concatenate((X_i_train_1, X_i_train_2, X_i_train_3, X_i_train_4), axis=0)
X_i_train = tf.convert_to_tensor(X_i_train, dtype=tf.float64)
# 创建 PINN 模型并进行训练
Layers = [2, 40, 40,40,1]  # 神经网络层结构
print(X_u_train)
model = HeatPINN(Layers)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005))
model.train(X_u_train, u_train, X_i_train, epochs=1000)
n_pred = 5000  # 预测集中的点数量

x_pred = np.random.uniform(low=0, high=1, size=(n_pred, 2))

# 将预测集转换为 tf.Tensor 类型
x_pred_tf = tf.convert_to_tensor(x_pred, dtype=tf.float64)

# 使用模型预测
y_pred = model.predict(x_pred_tf)

# 将预测结果转换为 numpy 数组类型（如果必要）
file_path ="D:\Desktop\graduate paper\hit pinn data_pred.npz"
np.savez(file_path, x_pred=x_pred, y_pred=y_pred)
                  #对于每个训练点，您可以通过为其分配一个权重来实现点点自适应性方法。在训练过程中，可以根据每个训练点的损失值来调整权重。较高的损失值对应着较高的权重。

#上述示例中，我添加了两个新的变量 `u_weights` 和 `f_weights`，它们分别用于存储内部节点和边界节点的权重。这些权重在每个训练步骤中都会被更新。

#在 `train_step` 方法中，我将 `u_weights` 和 `f_weights` 作为参数传递给 `loss_U` 和 `loss_PDE` 方法。这些方法会根据权重计算相应的损失值。

#在每个训练步骤的末尾，我使用 `tape.gradient()` 方法计算总损失相对于权重的梯度。然后，通过取梯度的绝对值并正则化，将权重归一化为和为1。



[[0.70621634 0.57009855]
 [0.91500864 0.91346293]
 [0.44191336 0.39962539]
 ...
 [0.36222247 0.56075248]
 [0.69595732 0.18451741]
 [0.59026972 0.43176907]]
Training loss (every 50 epochs) at epoch 50 : 8.236182880424062
Training loss (every 50 epochs) at epoch 100 : 0.89104511807715847
Training loss (every 50 epochs) at epoch 150 : 0.29318074110705072
Training loss (every 50 epochs) at epoch 200 : 0.14222797383665073
Training loss (every 50 epochs) at epoch 250 : 0.061878948431859829
Training loss (every 50 epochs) at epoch 300 : 0.042627981595831692
Training loss (every 50 epochs) at epoch 350 : 0.03175740628828029
Training loss (every 50 epochs) at epoch 400 : 0.028794788615313996
Training loss (every 50 epochs) at epoch 450 : 0.031234050263280643
Training loss (every 50 epochs) at epoch 500 : 0.024748962232286089
Training loss (every 50 epochs) at epoch 550 : 0.086965777926048121
Training loss (every 50 epochs) at epoch 600 : 0.021267533804328428
Training loss (every 50 epochs) at e